# Exercise: Performance Optimization

Optimize the following function.

In [1]:
function work!(A)
    val = zero(eltype(v))
    for i in 1:N
        val = mod(v[i],256)
        A[i,1:N] = B[i,1:N] * (sin(val) * sin(val) - cos(val) * cos(val))
    end
    A = A/2
    return A
end

work! (generic function with 1 method)

The following data is **fixed** and **not supposed to be modified**!

In [2]:
# do not modify this cell!

using Random
Random.seed!(42)

N = 250
B = rand(N,N)
v = rand(Int, N);

const result = work!(zeros(N,N));

# do not modify this cell!

You can compare against `result` to test your implementation(s):

In [3]:
using Test

@test work!(zeros(N,N)) ≈ result

Test Passed

You can benchmark as follows:

In [4]:
using BenchmarkTools

@btime work!(A) setup=(A=zeros(N,N)); # or use @benchmark for more information

  549.768 μs (3253 allocations: 1.54 MiB)


## Your Optimizations

Your optimized variants go here!

<details>

<summary><b>If you want some hints, click here!</b></summary>

* Try to look for type instabilities (with `@code_warntype`).
* Try to avoid unnecessary allocations (due to slicing and allocating array computations).
* Try to optimize the memory access (keyword: column-major order).
* Bonus: Simplify the algebra by using a trigonometric identity.

</details>

### Fixing the type instability (accessing global `N`)

In [5]:
@code_warntype work!(zeros(N,N))

MethodInstance for work!(::Matrix{Float64})
  from work!(A) @ Main In[1]:1
Arguments
  #self#::Core.Const(work!)
  A@_2::Matrix{Float64}
Locals
  @_3::Any
  val::Any
  i::Any
  A@_6::Matrix{Float64}
Body::Matrix{Float64}
1 ─       (A@_6 = A@_2)
│   %2  = Main.eltype(Main.v)::Any
│         (val = Main.zero(%2))
│   %4  = (1:Main.N)::Any
│         (@_3 = Base.iterate(%4))
│   %6  = (@_3 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_3::Any
│         (i = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Any
│   %12 = Base.getindex(Main.v, i)::Any
│         (val = Main.mod(%12, 256))
│   %14 = Main.B::Any
│   %15 = i::Any
│   %16 = (1:Main.N)::Any
│   %17 = Base.getindex(%14, %15, %16)::Any
│   %18 = Main.sin(val)::Any
│   %19 = Main.sin(val)::Any
│   %20 = (%18 * %19)::Any
│   %21 = Main.cos(val)::Any
│   %22 = Main.cos(val)::Any
│   %23 = (%21 * %22)::Any
│   %24 = (%20 - %23)::Any
│   %25 = (%17 * %24)::Any
│   %26 = A@_6::Matrix{Float64

Get `N` from the size of `A` (or `B`) or add another function argument.

In [6]:
function work1!(A, B, v)
    N = size(A,1) # or additional function argument
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    A = A/2
    return A
end

@btime work1!(A, $B, $v) setup=(A=zeros(N,N))
@test work1!(zeros(N,N), B, v) ≈ result

  378.110 μs (502 allocations: 1.48 MiB)


Test Passed

### Analytic optimization (style points 😉)

Trigonometric identity

In [7]:
x = rand()
@test sin(x) * sin(x) - cos(x) * cos(x) ≈ -cos(2*x)

Test Passed

In [8]:
function work2!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    A = A/2
    return A
end

@btime work2!(A, $B, $v) setup=(A=zeros(N,N))
@test work2!(zeros(N,N), B, v) ≈ result

  378.445 μs (502 allocations: 1.48 MiB)


Test Passed

### Avoid allocations due to slicing and allocating array arithmetic

In [9]:
function work3_vectorized!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))*0.5 # moved the rescaling by 0.5 here
        @views A[i,1:N] .= B[i,1:N] .* val # using @views and broadcasting here
    end
    return A
end

@btime work3_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_vectorized!(zeros(N,N), B, v) ≈ result

  99.691 μs (0 allocations: 0 bytes)


Test Passed

Same idea but explicit loop

In [10]:
function work3_loop!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))*0.5
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val
        end
    end
    return A
end

@btime work3_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_loop!(zeros(N,N), B, v) ≈ result

  89.115 μs (0 allocations: 0 bytes)


Test Passed

### Separating `val` computation

In [11]:
function work4_vectorized!(A, B, v)
    N = size(A,1)
    val = @. -cos(2*mod(v,256))*0.5
    
    for i in 1:N
        @views A[i,1:N] .= B[i,1:N] .* val[i]
    end
    return A
end

@btime work4_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_vectorized!(zeros(N,N), B, v) ≈ result

  93.329 μs (1 allocation: 2.06 KiB)


Test Passed

Same idea but explicit loop

In [12]:
function work4_loop!(A, B, v)
    N = size(A,1)
    val = @. -cos(2*mod(v,256))*0.5
    
    for i in 1:N
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work4_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_loop!(zeros(N,N), B, v) ≈ result

  84.571 μs (1 allocation: 2.06 KiB)


Test Passed

### Switch loop order (!!!)

In [13]:
function work5!(A, B, v)
    N = size(A,1)
    val = @. -cos(2*mod(v,256))*0.5
    
    for j in 1:N
        for i in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work5!(A, $B, $v) setup=(A=zeros(N,N))
@test work5!(zeros(N,N), B, v) ≈ result

  32.790 μs (1 allocation: 2.06 KiB)


Test Passed

And here with broadcasting

In [14]:
function work5_broadcasting!(A, B, v)
    val = @. -cos(2*mod(v,256))*0.5
    @. A = B * val
    return A
end

@btime work5_broadcasting!(A, $B, $v) setup=(A=zeros(N,N))
@test work5_broadcasting!(zeros(N, N), B, v) ≈ result

  32.722 μs (1 allocation: 2.06 KiB)


Test Passed

## Bonus Question: Performance limit?

Look at your final optimized version of `work!`.

* In the limit of larger `A` and `B`, what is conceptually limiting the performance, the compute capability or memory transfer (i.e. reading and writing `A` and `B`)?